In [8]:
# ---
# 01_data_loading_exploration.ipynb
# Sandbox: Data Loading & Initial Exploration
# ---

In [9]:
%pip install xlrd pandas

Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd

In [11]:
pd.set_option('display.max_columns', None)

In [12]:
DATA_PATH = '../data/bronze/HubSpot CRM All Contacts Feb 27 2025.xls'

In [15]:
view_head = False
try:
    df = pd.read_excel(DATA_PATH)
    n_cols = df.shape
    n_rows = df.shape[0]
    print("Dataset loaded successfully with {} rows and {} columns.".format(n_rows, n_cols[1]))
    view_head = True
except Exception as e:
    print(f"Error loading dataset: {e}")

Error loading dataset: [Errno 2] No such file or directory: '../data/bronze/HubSpot CRM All Contacts Feb 27 2025.xls'


In [14]:
cols = list(df.columns)
print("COLUMNS FOR DATASET")
print("--------------------")
for col in cols:
    print(col)

NameError: name 'df' is not defined

In [ ]:
sensitive_columns = [
    # Personally identifiable information
    'First Name', 'Last Name', '¿Cuál es tu nombre completo?', 'Apellido paterno',
    'Email Domain', 'Phone number', 'Phone Number', 'Mobile Phone Number',
    'Date of birth', 'Contraseña',
    
    # Postal codes
    'Código Postal', 'Postal Code',
    
    # IP data (can be traceable)
    'IP City', 'IP State/Region', 'IP Country', 'IP Timezone', 'IP Country Code', 'IP State Code/Region Code',
    
    # Personal documents
    'Documento Identificacion Frente', 'Documento Identificacion Reverso',
    'INE Anverso', 'INE Reverso', 'ine_pasaporte',
    'Comprobante Domicilio Anverso', 'Comprobante Domicilio Reverso',
    'Factura Original Anverso', 'Factura Original Reverso',
    
    # Personal marketing identifiers
    'Google ad click id', 'Facebook click id',
    
    # Other personal identifiers
    'Member email', 'Email of the agent that of the agent involved in the last WhatsApp conversation synced',
    'Email of the agent that of the agent involved in the last WhatsApp message sent from the HubSpot UI',
    'Domain to which registration email was sent', 'Inbox URL', 'LinkedIn URL'
]

In [ ]:
df_sanitized = df.drop(columns=sensitive_columns)

In [ ]:
if view_head:
    display(df_sanitized.head())

In [ ]:
# Identify columns where all values are null
null_columns = df_sanitized.columns[df_sanitized.isna().all()].tolist()

print(f"Number of columns with all values as NaN: {len(null_columns)}")
print(f"Columns with all values as NaN:")
for col in null_columns:
    print(f"- {col}")

# Drop all columns with all null values
df_sanitized_clean = df_sanitized.drop(columns=null_columns)

# Print the shape before and after dropping
print(f"\nDataFrame shape before dropping null columns: {df_sanitized.shape}")
print(f"DataFrame shape after dropping null columns: {df_sanitized_clean.shape}")
print(f"Number of columns removed: {df_sanitized.shape[1] - df_sanitized_clean.shape[1]}")

In [ ]:
# 1. Identify columns with more than 50% missing values
high_null_cols = df_sanitized_clean.columns[df_sanitized_clean.isna().mean() > 0.5].tolist()
print(f"Columns with more than 50% null values: {len(high_null_cols)}")
for col in high_null_cols:
    print(col)

In [ ]:
# 3. Create a cleaned DataFrame by dropping those columns
df_refined = df_sanitized_clean.drop(columns=high_null_cols)

In [ ]:
# 4. Show original and final shape
print(f"\nOriginal shape: {df_sanitized_clean.shape}")
print(f"Shape after dropping high-null columns: {df_refined.shape}")
print(f"Number of columns removed: {df_sanitized_clean.shape[1] - df_refined.shape[1]}")

In [ ]:
# 6. Display null percentage in remaining columns (top 10)
remaining_null_pct = df_refined.isna().mean().sort_values(ascending=False) * 100
print("\nNull percentage in remaining columns (top 10):")
print(remaining_null_pct.head(10))

In [ ]:
hubspot_marketing_columns = [
    'Record ID',
    'Average Pageviews',
    'Contact owner',
    'Contact unworked',
    'Create Date',
    'Cumulative time in "Lead (Lifecycle Stage Pipeline)" (HH:mm:ss)',
    'Currently in workflow',
    'Date entered "Lead (Lifecycle Stage Pipeline)"',
    'Date entered "Marketing Qualified Lead (Lifecycle Stage Pipeline)"',
    'Date exited "Lead (Lifecycle Stage Pipeline)"',
    'Date of first engagement',
    'Description of first engagement',
    'Event Revenue',
    'First Conversion',
    'First Conversion Date',
    'First marketing email open date',
    'First marketing email send date',
    'First Page Seen',
    'First Referring Site',
    'HubSpot Team',
    'ID of first engagement',
    'Last Activity Date',
    'Last Contacted',
    'Last marketing email name',
    'Last marketing email open date',
    'Last marketing email send date',
    'Last Modified Date',
    'Last Page Seen',
    'Last Referring Site',
    'Last Touch Converting Campaign',
    'Latest time in "Lead (Lifecycle Stage Pipeline)" (HH:mm:ss)',
    'Latest Traffic Source',
    'Latest Traffic Source Date',
    'Latest Traffic Source Drill-Down 1',
    'Latest Traffic Source Drill-Down 2',
    'Lead response time (HH:mm:ss)',
    'Lead Status',
    'Lifecycle Stage',
    'Marketing contact status',
    'Marketing contact until next update',
    'Marketing emails delivered',
    'Marketing emails opened',
    'Number of event completions',
    'Number of Form Submissions',
    'Number of Pageviews',
    'Number of Sales Activities',
    'Number of Sessions',
    'Number of times contacted',
    'Number of Unique Forms Submitted',
    'Original Traffic Source',
    'Original Traffic Source Drill-Down 1',
    'Original Traffic Source Drill-Down 2',
    'Owner assigned date',
    'Recent Conversion',
    'Recent Conversion Date',
    'Record source'
]

In [ ]:
# Drop hubspot_marketing_columns
df_refined = df_refined.drop(columns=hubspot_marketing_columns)

In [ ]:
df_refined.shape

In [ ]:
df_refined.head()

In [ ]:
null_percentage = df_refined.isna().mean().sort_values(ascending=False) * 100
print("\nNull percentage in remaining column")
print(null_percentage)

In [ ]:
def show_unique_values(df, max_unique=20, max_examples=5):
    results = []
    
    for col in df.columns:
        # Get non-null values
        non_null_values = df[col].dropna()
        
        # Calculate number of unique values
        n_unique = df[col].nunique()
        
        # Get counts of unique values
        value_counts = df[col].value_counts(dropna=False)
        
        # Determine data type
        dtype = df[col].dtype
        
        # Prepare examples of values
        if n_unique <= max_unique:
            # If there are few unique values, show all with their counts
            examples = value_counts.to_dict()
        else:
            # If there are many unique values, show only the first examples
            examples = non_null_values.head(max_examples).tolist()
        
        # Add information to results
        results.append({
            'Column': col,
            'Type': dtype,
            'Unique_Values': n_unique,
            'Nulls_%': null_percentage[col],
            'Examples': examples
        })
    
    return pd.DataFrame(results)

In [ ]:
# Show summary of unique values for each column
unique_values_summary = show_unique_values(df_refined)

In [ ]:
print("\n===== SUMMARY OF UNIQUE VALUES BY COLUMN =====")
for i, row in unique_values_summary.iterrows():
    print(f"\n{i+1}. {row['Column']} ({row['Type']})")
    print(f"   - Unique values: {row['Unique_Values']}")
    print(f"   - Nulls: {row['Nulls_%']:.2f}%")
    
    # Show examples of values
    print("   - Examples:")
    if isinstance(row['Examples'], dict):
        # If we have a dictionary of counts, show value: count
        for val, count in row['Examples'].items():
            if pd.isna(val):
                print(f"     * NaN: {count}")
            else:
                print(f"     * {val}: {count}")
    else:
        # If we have a list of examples
        for val in row['Examples']:
            print(f"     * {val}")